# Basics of Geographic Information Systems

## 0 Imports and Paths

The following packages will be needed for this notebook.

In [1]:
# Imports:
import requests
from shapely.geometry import shape, Point, LineString
import mplleaflet
import geopandas as gpd
from matplotlib import pyplot as plt
import os
from datetime import datetime

Paths for in- and output files:

In [2]:
# INPUT OPTIONS
IN_DIR = os.path.abspath("./data")
RESEARCH_ASSOCIATE_TRACK_FILENAME = "research_associate_moped_tour.geojson"

## 1 Introduction

This task aims at making you familiar with basic GIS functions. As you know from the theoretical part, there are different GIS functions, that are useful when working with geospatial data. This exercise covers the following functions:

* Geocoding
* Routing
* Reverse Geocoding
* Map-Matching

At the Institute of Automotive Technology (FTM), we use a GIS-API named "MAGIS", which is developed and hosted by the group "Smart Mobility" of FTM. This API integrates serveral open source GIS services and standardizes inputs and outputs (e.g. order of latitude and longitude in coordinates). All services are based on OSM and the API supports the OSM-region for europe.

Further information can be accessed via the following URLs:

| Description                            | URL           |
|:-------------------------------------  |:------------- |
| quick help and example queries         | [https://wiki.tum.de/display/smartemobilitaet/Schnellzugriff](https://wiki.tum.de/display/smartemobilitaet/Schnellzugriff) |
| documentation of the architecture      | [https://wiki.tum.de/display/smartemobilitaet/Allgemeine+Architektur](https://wiki.tum.de/display/smartemobilitaet/Allgemeine+Architektur) |
| detailed documentation of each service | [https://wiki.tum.de/display/smartemobilitaet/Services](https://wiki.tum.de/display/smartemobilitaet/Services) |

## 2 GIS Services

MAGIS is available via the following URL:

In [3]:
## OPTIONS
MAGIS_BASE_URL = "https://fdu.ftm.mw.tum.de/gis"

### 2.1 Geocoding

Imagine you are at university (origin: "Garching Forschungszentrum") and you want to go to Marienplatz (destination: "Marienplatz, München") by car. In order to use a routing engine, you fist have to find out the coordinates of those locations. So, how can you find out the coordinates of your origin and destination just by the name of the locations?

In order to get map elements by their name, you use the geocode function, which takes a search text as input and returns map elements as output. Thats why we first save the search strings of the locations in the following variables:

In [4]:
ORIGIN = "Garching Forschungszentrum"
DESTINATION = "Marienplatz, München"

In the following code block, we already prepared a template query as a string variable and assembled the geocode query for your origin. The relevant parameters for the query are:

* q / query_text: The search string you would type into a text field (e.g. on google maps).
* country_codes: Restricts your search results to the specified countries (e.g. "de" for Germany), coutry codes from ISO 3166-1
* limit / limit: Limits the number of results to the specified maximum.

In [5]:
# Build geocode template
geocode_query_template = "/geocode?q={query_text}&countrycodes=de&limit={limit}"
# Build origin query
geocode_origin_query = MAGIS_BASE_URL + geocode_query_template.format(query_text=ORIGIN,limit=1)

Now, assemble the geocode query for your destination location in the variable geocode_destination_query and print both queries:

In [6]:
# Build destination query
#<<solution>>
geocode_destination_query = MAGIS_BASE_URL + geocode_query_template.format(query_text=DESTINATION, limit=1)

# Print queries for testing purposes
print(geocode_origin_query)
print(geocode_destination_query)
#<</solution>>

http://gis.ftm.mw.tum.de/geocode?q=Garching Forschungszentrum&countrycodes=de&limit=1
http://gis.ftm.mw.tum.de/geocode?q=Marienplatz, München&countrycodes=de&limit=1


You may copy one of the query URLs and paste it in your browser's address field to manually look at the results.

Since we have assembled the query URLs for the MAGIS, we now need to send the request to the API. We use the "requests" library to query the API:

In [7]:
# Get responses
response_geocode_origin = requests.get(geocode_origin_query)
response_geocode_destination = requests.get(geocode_destination_query)

The responses include one json element respectively, which can be extracted by the function "you_request_response.json()":

In [8]:
# Get Jsons
geocode_origin_json = response_geocode_origin.json()
geocode_destination_json = response_geocode_destination.json()

In order to work with the latitude and longitude, we need to extract the relevant coordinates from the json-object. Those coordinates are stored in the first "features" element within the "geometry" element and are named "coordinates". After extracting the coordinates, we are able to store them as Point objects. 

In [9]:
# Extract coordinates from json object:
origin_coordinates = geocode_origin_json["features"][0]["geometry"]["coordinates"]
destination_coordinates = geocode_destination_json["features"][0]["geometry"]["coordinates"]
# Print coordinates
print(origin_coordinates)
print(destination_coordinates)
# Convert response to Points
origin = Point(origin_coordinates)
destination = Point(destination_coordinates)

[11.6712796, 48.2649836]
[11.5763926, 48.1368802]


If you paste those coordinates into Google Maps, can you confirm that the geocoding was successful?

Let's have a look at the results of the geocoding and plot our origin and destination on a map with mplleaflet:

In [10]:
# Let's check the geo-coding result
fig = plt.figure()
plt.scatter(*origin.xy)
plt.scatter(*destination.xy)
mplleaflet.display(fig=fig)

### 2.2 Routing

As a second step, we are going to do a routing between origin and destination. Therefore, once again define a template string for the routing service and assemble the query URL:

In [11]:
## ROUTE BETWEEN ORIGIN AND DESTINATION
route_query_template = "/route?coordinates={coordinate_list}"
coordinate_list = [origin.x, origin.y, destination.x, destination.y]
route_query = MAGIS_BASE_URL + route_query_template.format(coordinate_list=str(coordinate_list))

For testing purposes, we print the query. Once again, you may copy the query and paste it in your browser to look at the results.

In [12]:
# Print query for testing purposes
print(route_query)

http://gis.ftm.mw.tum.de/route?coordinates=[11.6712796, 48.2649836, 11.5763926, 48.1368802]


As we already learned during geocoding, we have to send the request to the API, extract the json object and then continue processing the results. In this case, we generate a linestring from the route's geometry.

In [13]:
response_routing = requests.get(route_query)
routing_json = response_routing.json()

# Of all alternative routes, take the first one:
route = routing_json["routes"][0]
 
# Make a LineString from the route's geometry
route_line = LineString(route["geometry"]["coordinates"])

In order to check our results so far, we draw the points (origin and destination) as well as the route as a linestring on a map:

In [14]:
# Let's check the routing result
fig = plt.figure()
plt.scatter(*origin.xy)
plt.scatter(*destination.xy)
plt.plot(*route_line.xy)
mplleaflet.display(fig=fig)

### 2.3 Reverse Geocoding

Just before you start in Garching, your nerdy friend Sheldon sends you a text message:

"Hey there!\
I'm inviting some people for a games night. Wanna come around?\
My apartment is here: (48.179318, 11.553664). Be there at 7 p.m.\
cu"


Because you don't know, where Sheldon's apartment is and you don't want to drive a big detour, you first want to find out, where these coordinates are. First of all, you store his coordintes in a list:

In [15]:
STOPOVER = [11.553664, 48.179318]

In order to find out, where he lives, use the reverse geocoding service!

Hint: Follow these steps:

* Build a template query in the variable "reverse_geocode_query_template"
* Assemble the query string with the stopover coordinates in a variable named "reverse_query"
* Print the reverse_query for testing purposes


* Execute the request (variable: "response_reverse_geocoding")
* Extract the json object (variable: "reverse_geocoding_json")


* Extract the name of the location (save it in the variable "stopover_name"): features -> 0 -> properties -> name

In [16]:
# Build template query: reverse_geocode_query_template
#<<solution>>
reverse_geocode_query_template = "/reverse?coordinates={coordinate_list}"
#<</solution>>

# Assemble query for stopover: reverse_query
#<<solution>>
reverse_query = MAGIS_BASE_URL + reverse_geocode_query_template.format(coordinate_list=str(STOPOVER))
#<</solution>>

# Print reverse_query
#<<solution>>
print(reverse_query)
#<</solution>>

http://gis.ftm.mw.tum.de/reverse?coordinates=[11.553664, 48.179318]


In [17]:
# Execute the request and save the results in: response_reverse_geocoding
#<<solution>>
response_reverse_geocoding = requests.get(reverse_query)
#<</solution>>

# Extract json object from results and save it as: reverse_geocoding_json
#<<solution>>
reverse_geocoding_json = response_reverse_geocoding.json()
#<</solution>>

In [18]:
# Extract the name of the location and save it as stopover_name
#<<solution>>
stopover_name = reverse_geocoding_json["features"][0]["properties"]["name"]
#<</solution>>

In [19]:
# Print stopover name
print("The stopover is: {}".format(stopover_name))

The stopover is: Studentendorf Oberwiesenfeld


Since you know, where Sheldon lives, you decide to visit him and to join the games night at his apartment on your way home. Therefore, you recalculate your route with a new coordinate_list.

* Update the coordinate_list
* Assemble the routing URL: stopover_route_query (for testing purposes, you can print the URL once again)
* Execute the routing request: response_stopover_route
* Extract the json object: stopover_route_json
* Extract the first route of the returned list of routes
* Create a LineString from the returned coordinates (route -> geometry -> coordinates): stopover_route_line

In [20]:
# Update the variable coordinate_list: coordinate_list
#<<solution>>
coordinate_list = [origin.x, origin.y, STOPOVER[0], STOPOVER[1], destination.x, destination.y]
#<</solution>>


# Assemble the routing URL: stopover_route_query
#<<solution>>
stopover_route_query = MAGIS_BASE_URL + route_query_template.format(coordinate_list=str(coordinate_list))
#<</solution>>

# Print the URL:
#<<solution>>
print(stopover_route_query)
#<</solution>>


# Execute the routing request: response_stopover_route
#<<solution>>
response_stopover_route = requests.get(stopover_route_query)
#<</solution>>

# Extract the json object: stopover_route_json
#<<solution>>
stopover_route_json = response_stopover_route.json()
#<</solution>>

# Extract the first route of the returned list of routes
#<<solution>>
stopover_route = stopover_route_json["routes"][0]
#<</solution>>


# Create a LineString from the returned coordinates (route -> geometry -> coordinates): stopover_route_line
#<<solution>>
stopover_route_line = LineString(stopover_route["geometry"]["coordinates"])
#<</solution>>

http://gis.ftm.mw.tum.de/route?coordinates=[11.6712796, 48.2649836, 11.553664, 48.179318, 11.5763926, 48.1368802]


Finally, let's visualize your results on a map including your new route as well as the points of your origin, your stopover, and you destination:

In [21]:
# Let's check the routing result
fig = plt.figure()
plt.scatter(*origin.xy)
plt.scatter(*destination.xy)
plt.scatter(*Point(STOPOVER).xy)
plt.plot(*stopover_route_line.xy)
mplleaflet.display(fig=fig)

### 2.4 Map-Matching

The next day, you go to university and have a meeting with your supervisor for a student research project. Your supervisor tells you, that he was driving with his moped the day before and that you could have met on your way from Sheldon's apartment to Marienplatz after the games night. He provides a geojson file with his recorded track:

In [22]:
# Load Research Associate Tour
research_associate_tour = gpd.read_file(os.path.join(IN_DIR,RESEARCH_ASSOCIATE_TRACK_FILENAME), driver ='GeoJSON')

# Show the data included in his file:
research_associate_tour

,track_fid,track_seg_id,track_seg_point_id,ele,time,geometry
0,0,0,0,562.0,2019-05-26T16:39:26,POINT (11.585737 48.17532)
1,0,0,1,561.0,2019-05-26T16:39:52,POINT (11.585615 48.17527)
2,0,0,2,559.0,2019-05-26T16:39:56,POINT (11.58556 48.175404)
3,0,0,3,559.0,2019-05-26T16:39:59,POINT (11.585569 48.175507)
4,0,0,4,557.0,2019-05-26T16:40:03,POINT (11.585573 48.175613)
5,0,0,5,555.0,2019-05-26T16:40:08,POINT (11.585558 48.175713)
6,0,0,6,555.0,2019-05-26T16:40:13,POINT (11.585533 48.175827)
7,0,0,7,552.0,2019-05-26T16:40:16,POINT (11.585524 48.175934)
8,0,0,8,553.0,2019-05-26T16:40:19,POINT (11.585516 48.176056)
9,0,0,9,553.0,2019-05-26T16:40:21,POINT (11.585485 48.176178)


In [23]:
# Let's plot his tour on a map to see where he drove
ax = research_associate_tour.plot()
mplleaflet.display(fig=ax.figure)

As you can see, this is a pretty noisy track and we can only visually see which streets your supervisor used but our program is not aware of the streets he traveled. That's why we need some map matching here in order to compare his route to yours.

Map matching services take free GPS points as an input and return "snapped" GPS points, that lie on existing roads. Some map matching algoriths additionally return the map elements (nodes), that you passed.

Since GPS inaccuracy may be greater than the width of a road, you would not be able to determine, in which direction the track is following a road. Thus, map matching algorithms have to take into account more than one GPS point to identify the right road and the right direction.

Map matching algorithms work even better when considering timestamps for each GPS point: The algorithm then knows, how long it took to get from one GPS point to another and this allows verifying, if the calculated distance is valid.

That's why we have to provide both a list of coordinates as well as a list of timestamps in our template query:

In [24]:
# Map Matching Template Query
mapmatch_query_template = "/match?coordinates={coordinate_list}&time={time_list}"

MAGIS works with timestamps that comply with the ISO 8601 standard (e.g. 2019-05-26 16:40:00). As you can see above in the printed extract of the dataframe, the timestamp column doesn't match the mandatory format. Furthermore, it was only read in as a string column and the column isn't recognized as timestamps. As a first step we use a lambda function, that converts the strings into datetime objects considering the input format:

In [25]:
# Declare input and output format
input_time_format = '%Y-%m-%dT%H:%M:%S'
output_time_format = '%Y-%m-%d %H:%M:%S'

# Converting timestamps
research_associate_tour.time = research_associate_tour.time.apply(func=lambda x: datetime.strptime(x, input_time_format))

Since the input for the map matching service is a URL including a coordinate_list and a corresponding timestamp_list, we have to extract the time values in the right format and store it to a list (without the apostrophes).

In [26]:
# Extract the list of timestamps following the output format
time_list = research_associate_tour.time.apply(func=lambda x: x.strftime(output_time_format)).values
# Convert the output to a string and remove all apostrophes
time_list_str = str(list(time_list)).replace("'","")

Now we have to extract the coordinate_list: First convert the coordinates to tupel with x,y and extract the values. Then output the coordinates to one list:

In [27]:
coordinate_list = research_associate_tour.geometry.apply(lambda x: (x.x, x.y))
coordinate_list = coordinate_list.values
coordinate_list = [coordinate for coordinate_tuple in coordinate_list for coordinate in coordinate_tuple]

Now we can assemble the URL and print it for testing purposes:

In [28]:
# Assemble map matching URL for MAGIS
mapmatch_research_associate_tour_query = MAGIS_BASE_URL + mapmatch_query_template.format(coordinate_list = coordinate_list,time_list=time_list_str)

# Print map matching query for testing purposes
print(mapmatch_research_associate_tour_query)

http://gis.ftm.mw.tum.de/match?coordinates=[11.585737, 48.17532, 11.585615, 48.17527, 11.58556, 48.175404, 11.585569, 48.175507, 11.585573, 48.175613, 11.585558, 48.175713, 11.585533, 48.175827, 11.585524, 48.175934, 11.585516, 48.176056, 11.585485, 48.176178, 11.585465, 48.176327, 11.585433, 48.176483, 11.585399, 48.176655, 11.585387, 48.176811, 11.585377, 48.176941, 11.585372, 48.177055, 11.585397, 48.177177, 11.585426, 48.177265, 11.585469, 48.177399, 11.585507, 48.177532, 11.585519, 48.177677, 11.585471, 48.177841, 11.585369, 48.178017, 11.585318, 48.178116, 11.585279, 48.178215, 11.585264, 48.178314, 11.58526, 48.178425, 11.585238, 48.178532, 11.585236, 48.178654, 11.58524, 48.178768, 11.585229, 48.178894, 11.585203, 48.179028, 11.585191, 48.179157, 11.585181, 48.179279, 11.585166, 48.179401, 11.585146, 48.179516, 11.585131, 48.179634, 11.585113, 48.179741, 11.585108, 48.179871, 11.585093, 48.179993, 11.585077, 48.180115, 11.585063, 48.180244, 11.585076, 48.180382, 11.585067, 48.1

Now we execute the request and process the results:

In [29]:
# Execute the map matching request
response_mapmatch_research_associate_tour = requests.get(mapmatch_research_associate_tour_query)
# Extract the json object
mapmatch_research_associate_tour =  response_mapmatch_research_associate_tour.json()

In order to visualize the results, we need to insert the returned locations ("matched_locations") in a geodataframe. This is achieved by:

* extracting the matched locations from the json object
* converting the coordinates (X, Y) to Points (shapely objects)
* (extracting the matched nodes from the json object)
* filling the Points into a geopandas dataframe
* merging the Points to receive a LineString (shapely object).

In [30]:
# Convert Result to GeoDataFrame for research associate
matched_locations = mapmatch_research_associate_tour["matched_locations"]
matched_points = [Point(coordinates[0], coordinates[1]) for coordinates in matched_locations]
matched_nodes = mapmatch_research_associate_tour["nodes"]
research_associate_tour_gpd = gpd.GeoDataFrame(geometry=matched_points)
research_associate_tour_line = LineString(matched_points)

In [31]:
# Let's check the routing result
fig = plt.figure()
plt.plot(*research_associate_tour_line.xy)
mplleaflet.display(fig=fig)

Since we now have the map matched route of your supervisor and your own route (with the stopover at Sheldon's apartment), you can calculate the crossing point of the two routes. Save the result in a variable called "meeting_points".

**Hint:** Use a shapely function to find the corresponding point!

In [32]:
# Find intersection of stopover route and supervisor's route: meeting_points
#<<solution>>
meeting_points = research_associate_tour_line.intersection(stopover_route_line)
#<</solution>>

Finally, we visualize the results on a map to verify the calculations:

In [33]:
# Visualize the results on a map
figure=plt.figure()
plt.plot(*route_line.xy,linewidth=5.0)
plt.plot(*stopover_route_line.xy, color = 'red', linewidth = 5.0)
plt.plot(*research_associate_tour_line.xy, color='green', linewidth=5.0)
for pt in meeting_points:
    plt.scatter(*pt.xy, color='k', marker='x', s=100)
mplleaflet.display(fig=figure)